# Notebook to obtain tweets that talk about policy actions of COVID

**input data** : tweets de cuentas de gobiernos estatales y de secretaría de salud. 
**output data**: data frame con:
    - entidad
    - policy action
    - date
    - evidencia (tweet)
    - categoria de policy action
    
(Frist case Reported in México: 2020-02-28)

In [1]:
import pandas as pd
import re
import datetime
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [122]:
porter = PorterStemmer()
lancaster = LancasterStemmer()

In [123]:
data_tweets = pd.read_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/COVID-19 Mexico/Twitter/data/raw/data.csv')
input_data = data_tweets[['name_entidad', 'tipo_cuenta','username_twitter', 'id', 'date', 'hashtags', 'tweet' ]].dropna(subset = ['date'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [125]:
input_data[['date']] = input_data.apply(lambda x: datetime.datetime(int(re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', x['date'])[0].split('-')[0]), int(re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', x['date'])[0].split('-')[1]), int(re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', x['date'])[0].split('-')[2])), axis = 1)

In [61]:
## Filter tweets that talks about covid

In [126]:
keywords_data = pd.read_csv('../data/raw/keywords.csv')
keywords_data = keywords_data.dropna()
covid_words = ['coronavirus', 'covid19', 'covid']
all_words = list(keywords_data.keyword)

In [135]:
policy_words = ['accion', 'propuesta', 'politica', 
               'restriccion' ,'cuarentena', 'distanciamiento' ,'social', 
               'distancia', 'prevencion', 'medidas', 'monitoreo', 'evitar', 
               'contagios', 'informe', 'contingencia', 'coronavirus', 
               'covid19', 'covid', 'preparar', 'protocolo']

In [141]:
porter_keywords = []
for i in all_words:
    porter_keywords.append(porter.stem(i))
porter_covid = []
for i in covid_words:
    porter_keywords.append(porter.stem(i))
porter_policy = []
for i in policy_words:
    porter_policy.append(porter.stem(i))

In [129]:
def get_key_words(text, list_filter):
    tokens = word_tokenize(text)
    key_similar = []
    for i in tokens:
        l = i.lower()
        p = porter.stem(l)
        if p in list_filter or i in porter_covid:
            key_similar.append(i)
    
    if any(key_similar):
        return(key_similar)
    else:
        return('filter')

In [130]:
input_data['keywords_covid'] = input_data.apply(lambda x:get_key_words(x['tweet'], porter_keywords), axis = 1)

In [131]:
filter_keywords = input_data[input_data['keywords_covid'] != 'filter']

In [132]:
print(len(input_data), len(filter_keywords))

43567 18019


In [133]:
filter_keywords.groupby(['name_entidad']).count().head()

,tipo_cuenta,username_twitter,id,date,hashtags,tweet,keywords_covid
name_entidad,,,,,,,
aguascalientes,138,138,138,138,138,138,138
baja california,289,289,289,289,289,289,289
baja california sur,272,272,272,272,272,272,272
campeche,275,275,275,275,275,275,275
chiapas,273,273,273,273,273,273,273


In [ ]:
## Policy Actions

In [143]:
filter_keywords['dummy_policy'] = input_data.apply(lambda x:get_key_words(x['tweet'], porter_policy), axis = 1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [147]:
print(len(filter_keywords), len(filter_keywords[filter_keywords['dummy_policy'] != 'filter']))

18019 14919


In [145]:
filter_keywords.to_csv('../data/raw/tweets_policy_action.csv', encoding = 'utf-8 sig')

In [8]:
filter_keywords = pd.read_csv('../data/raw/keywords.csv')

In [9]:
filter_keywords

,keyword,category
0,multas,prohibiciones
1,cierre,prohibiciones
2,aforo,prohibiciones
3,actividad no esencial,prohibiciones
4,suspensión,prohibiciones
...,...,...
93,app,comuniciacion
94,micrositio,comuniciacion
95,denuncias,comuniciacion
96,cuestionario digital,comuniciacion


In [6]:
filter_keywords[filter_keywords['tipo_cuenta'].isna()]

,name_entidad,tipo_cuenta,username_twitter,id,date,hashtags,tweet,keywords_covid,keywords_policy,dummy_policy,Unnamed: 10,Unnamed: 11
138,NaN,NaN,NaN,1.260000e+18,5/15/20,"['#delicias', '#puertecitos', '#enbcnoscuidamo...",📍 #Delicias y #Puertecitos.\n#EnBCNosCuidamosT...,"['Coronavirus', 'COVID19']","['Coronavirus', 'COVID19']",NaN,NaN,NaN
139,NaN,NaN,NaN,1.260000e+18,5/15/20,['#saludencasa'],Muchas mujeres y hombres bajacalifornianos rec...,['acciones'],['acciones'],NaN,NaN,NaN
140,NaN,NaN,NaN,1.260000e+18,5/15/20,"['#quédateencasa', '#enbcnoscuidamostodos', '#...",Aumentaron 165 casos confirmados por Covid-19 ...,"['contagios', 'Coronavirus', 'COVID19']","['contagios', 'Coronavirus', 'COVID19']",NaN,NaN,NaN
141,NaN,NaN,NaN,1.260000e+18,5/15/20,"['#enbcnoscuidamostodos', '#coronavirus', '#co...",Sabemos que este es un momento muy difícil par...,"['Coronavirus', 'COVID19']","['Coronavirus', 'COVID19']",NaN,NaN,NaN
142,NaN,NaN,NaN,1.260000e+18,5/15/20,"['#enbcnoscuidamostodos', '#coronavirus', '#co...",Una forma de sumarte y ayudar a enfrentar esta...,"['medidas', 'Coronavirus', 'COVID19']","['medidas', 'Coronavirus', 'COVID19']",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15041,NaN,NaN,NaN,1.240000e+18,3/4/20,"['#ccco', '#oaxacalotienetodo']",2019 fue el año en que se celebró la unión en ...,['servicios'],filter,NaN,NaN,NaN
15042,NaN,NaN,NaN,1.240000e+18,3/4/20,"['#ccco', '#oaxacalotienetodo']","El mejor lugar para realizar congresos, conven...",['ferias'],filter,NaN,NaN,NaN
15043,NaN,NaN,NaN,1.240000e+18,3/4/20,"['#ccco', '#guelaguetza2019', '#oaxacalotienet...","El #CCCO, fue sede por pirmera vez de la Feria...",['Feria'],filter,NaN,NaN,NaN
15044,NaN,NaN,NaN,1.240000e+18,3/4/20,"['#ccco', '#oaxacalotienetodo']",En el Centro Cultural y de Convenciones de Oax...,['Informe'],['Informe'],NaN,NaN,NaN


In [108]:
filter_keywords.to_csv('../data/raw/twitter_keywords.csv')